### 解法  
###### 0の各マスにて、代入可能な数の候補数をそれぞれ求める
- a.   0のマス分、下記の動作を行う
- a-1. 同じ行、列、9x9マスに存在する数のリストを作成する
- a-2. a-1より、代入可能な数の候補数を出す
- a-3. 代入可能な数の候補数の最小値を出す
    - ------
###### 数字を代入する
- b.   0のマス分、下記の動作を行う
- b-1. マスが、aで出された候補数が最小値の場合、代入可能な数のリストからランダムで選出された数を代入する
- b-2. b-1の動作を2回まで行い、aに戻り再度繰り返す
    - ------
###### リセット、終了条件
- c.   代入可能な数がなくなった場合、aとbの動作をリセットし、最初入力された数を使い、再度aから動作を行う
- c.   全てのマスに0が存在しなくなった場合終了

In [3]:
import pandas as pd
import random
import copy
import numpy as np

In [15]:
# 数独の問題

# ↓解答データ
# http://www.sudokugame.org/archive/printsudoku.php?nd=4&xh=400
sudoku_1 =[[0,3,0,0,9,0,0,1,0],
           [1,0,9,0,8,0,0,0,6],
           [6,0,0,7,0,0,0,0,0],
           [0,0,1,0,0,0,8,0,0],
           [2,0,6,0,0,8,0,0,0],
           [0,0,0,0,0,3,0,4,0],
           [0,0,7,6,0,1,0,0,8],
           [0,0,2,0,0,0,0,3,4],
           [0,0,0,0,0,0,9,0,0]]

In [16]:
# バックアップを取る
sudoku_copy = copy.deepcopy(sudoku_1)

In [17]:
# 数独の盤面を表示する関数
def print_sudok(start):
    
    for raw_i in range(9):
        if ((raw_i == 3) or (raw_i == 6) or (raw_i == 9)):
            print('ー    ー    ー    ー    ー    ー    ー    ー    ー    ー')
    
        for col_i in range(9):
            print(start[raw_i][col_i], '    ',end='')

            if ((col_i == 2) or (col_i == 5)):
                print('|', '  ',end='')
        print('\n')

In [18]:
# 盤面を表示する
print_sudok(sudoku_1)

0     3     0     |   0     9     0     |   0     1     0     

1     0     9     |   0     8     0     |   0     0     6     

6     0     0     |   7     0     0     |   0     0     0     

ー    ー    ー    ー    ー    ー    ー    ー    ー    ー
0     0     1     |   0     0     0     |   8     0     0     

2     0     6     |   0     0     8     |   0     0     0     

0     0     0     |   0     0     3     |   0     4     0     

ー    ー    ー    ー    ー    ー    ー    ー    ー    ー
0     0     7     |   6     0     1     |   0     0     8     

0     0     2     |   0     0     0     |   0     3     4     

0     0     0     |   0     0     0     |   9     0     0     



In [19]:
#マス目が0が否か確認する
def check_0(num):
    rows = num//9 #numが0～8のとき、行は常に0になる
    columns = num%9 #numが0～8のとき、列は0～８に変動する
    
    if box[rows][columns] == 0:
        return True

In [20]:
#代入したマス目と同じ行、列に存在する数字を確認する関数
def check_queue(num):
    check_queue_box = []

    #同じ列の数字を確認
    for i in range(9):
        rows = i
        columns = num%9
        rows_columns = box[rows][columns]

        check_queue_box.append(rows_columns)
    
    #同じ行の数字を確認
    for j in range(9):
        rows = num//9
        columns = j
        rows_columns = box[rows][columns]

        check_queue_box.append(rows_columns)
    
    return check_queue_box


#同じ9x9マスに存在する数字を確認する関数
def check_9x9(num):
    check_9x9_box = []

    rows = num//9
    columns = num%9

    x = rows // 3*3
    y = columns // 3*3

    for i in range(0,3):

        for j in range(0,3):

        #上記の式の場合、行、列の数が9を超えるケースがあるため、下記で制限を加える
          if (x+i < 9) and (y+j < 9):
            l = box[x+i][y+j]
            check_9x9_box.append(l)

    return check_9x9_box


#同じ行、列、9x9マスに存在する数字のリストを作成する関数
def creat_list(h):
    check_queue_box = check_queue(h) #関数で返された、同じ行列の数字をリストに代入する
    check_9x9_box = check_9x9(h) #関数で返された、9同じ9x9マスの数字をリストに代入する
    check_queue_box.extend(check_9x9_box) #上記の行列、9x9マスの数字のリストを統合する 
    
    nums_list = set(check_queue_box)
    
    return nums_list


#代入可能な数の候補数を取得
def count_num_searching():
    count_num = []
    
    for h in range(81):

    #数字が0の場合、代入可能な数字の数をカウントする
    #数字が0でない場合は、0をカウントする
        if check_0(h):

            rows = h//9
            columns = h%9

            nums_list = creat_list(h)
            count_num.append(len(nums_list))
            
        
        elif not check_0(h):
            count_num.append(0)

     #カウントの最大値を取得
    max_count = max(count_num)
        
    #カウント最大値の要素番号を取得
    #検証用
    num_max_count = [i for i, x in enumerate(count_num) if x == max_count]
        
    return max_count, num_max_count


#代入可能な数字を探す関数
def input_nums_searching(num):
    nums_list = list(range(0,10))
    
    set_list = set(num) ^ set(nums_list)
    list_ = list(set_list)
    
    if len(list_) != 0:
        num_p = random.choice(list_)
    
    elif len(list_) == 0:
        num_p = 'none'
    
    return num_p

#代入可能な数の候補数が最小のマスを埋める関数
def num_input(max_count):
    count = 0
    for h in range(81):
        
        if check_0(h):
            rows = h//9
            columns = h%9
            
            nums_list = creat_list(h)
            
            if len(nums_list) == max_count:
        
                while count < 1:
        
                    num = input_nums_searching(nums_list)
                    box[rows][columns] = num
                    count = count + 1
        
                    break

In [21]:
print('----- 0の数-----')
q = 0
while q < 10000:
    nn = 0 #0の数をカウント
    oo = 0 #noneをカウント
    nn_tmp = []
    d = []
    box = copy.deepcopy(sudoku_copy)

    
    for i in range(81):
        nn = 0
        # 代入可能な候補数の最小値を取得
        max_count = count_num_searching()[0]
        
        #　最小値を用い、数字を代入する
        num_input(max_count)
        
        for mm in range(9):
            k = box[mm][0:9]
            j = k.count(0)
            o = k.count('none')

            nn = nn + j
            oo = oo + o
        #9x9マスの0のカウントを記録し、4つ以上の場合はリストdに入れる
        nn_tmp.append(nn)
        d = [e for e in nn_tmp if nn_tmp.count(e) > 4]
        
        #検証用
        #0が10個以下の場合出力
        if nn < 10:
            print(nn)
            print('-',q,'-')
        
            if nn < 5:
                print('close!')
                
        #0がなくなった場合、答えとして出力
        if nn < 1:
            print('correct!')
            answer = box
            print('-',q,'-')
            q = q+ 100000
            break
            
        #0カウントが4つ、もしくはnoneが存在するの場合は、breakし、リセットする
        if (len(d) > 4) or (oo > 0):
            d.clear()
            q = q + 1
            
            break
            
    else:
        continue
print('done')

----- 0の数-----
9
- 23 -
8
- 23 -
9
- 24 -
8
- 24 -
7
- 24 -
6
- 24 -
5
- 24 -
4
- 24 -
close!
3
- 24 -
close!
2
- 24 -
close!
1
- 24 -
close!
0
- 24 -
close!
correct!
- 24 -
done


In [22]:
print_sudok(answer)
# ↓解答
# http://www.sudokugame.org/archive/printsudoku.php?nd=4&xh=400

7     3     8     |   5     9     6     |   4     1     2     

1     2     9     |   3     8     4     |   7     5     6     

6     5     4     |   7     1     2     |   3     8     9     

ー    ー    ー    ー    ー    ー    ー    ー    ー    ー
3     4     1     |   9     2     7     |   8     6     5     

2     7     6     |   4     5     8     |   1     9     3     

9     8     5     |   1     6     3     |   2     4     7     

ー    ー    ー    ー    ー    ー    ー    ー    ー    ー
4     9     7     |   6     3     1     |   5     2     8     

5     1     2     |   8     7     9     |   6     3     4     

8     6     3     |   2     4     5     |   9     7     1     



In [23]:
#=======数独データ==========

In [ ]:
#問題2
# ↓解答
# http://www.sudokugame.org/archive/printsudoku.php?nd=4&xh=300
sudoku_2 =[[0,0,6,0,0,0,5,0,0],
           [0,0,0,0,2,4,0,0,0],
           [0,8,0,0,0,0,0,0,9],
           [4,0,0,0,0,3,0,0,0],
           [0,0,0,0,0,0,0,0,0],
           [0,0,0,0,0,0,6,2,5],
           [0,0,0,0,0,0,0,4,3],
           [0,0,0,0,0,0,0,0,0],
           [0,0,9,5,6,0,0,0,0]]

In [ ]:
#問題3
# ↓解答
# http://www.sudokugame.org/archive/printsudoku.php?nd=4&xh=100
sudoku_3 =[[0,0,0,0,0,9,7,0,0],
           [0,0,4,0,0,0,0,0,0],
           [0,0,2,0,0,0,0,0,8],
           [0,0,0,2,6,0,0,0,3],
           [0,9,1,0,0,0,0,0,0],
           [0,7,0,0,0,0,0,0,0],
           [0,0,0,8,0,0,0,0,2],
           [0,1,0,0,0,0,9,0,0],
           [0,0,8,3,0,0,0,0,0]]